In [25]:
import pandas as pd


def load_dataset(dataname):
    train = pd.read_csv('{}/train.tsv'.format(dataname),
                        sep='[\t\f\r\v]', names=['query1', 'query2', 'label'])

    valid = pd.read_csv('{}/dev.tsv'.format(dataname),
                        sep='\t', names=['query1', 'query2', 'label'])

    test = pd.read_csv('{}/test.tsv'.format(dataname),
                       sep='\t', names=['query1', 'query2', 'label'])

    return train, valid, test


name = 'lcqmc'
train, valid, test = load_dataset(name)

C:\Users\99480\AppData\Local\Temp\ipykernel_10128\2363752787.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train = pd.read_csv('{}/train.tsv'.format(dataname),


In [26]:
import jieba

for index, dataset in enumerate([train, valid, test]):
    if index != 2:
        dataset.dropna(axis=0, subset='label', inplace=True)
    dataset['cut_q1'] = dataset.apply(
        lambda row: jieba.lcut(row['query1']) if row['query1'] is not None else [], axis=1)
    dataset['cut_q2'] = dataset.apply(
        lambda row: jieba.lcut(row['query2']) if row['query2'] is not None else [], axis=1)

In [27]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import numpy as np

"""
LineSentence(inp)：格式简单：一句话=一行; 单词已经过预处理并被空格分隔。,inp 为数据路径
    size：是每个词的向量维度；
    window：是词向量训练时的上下文扫描窗口大小，窗口为5就是考虑前5个词和后5个词；
    min-count：设置最低频率，默认是5，如果一个词语在文档中出现的次数小于5，那么就会丢弃；
    workers：是训练的进程数（需要更精准的解释，请指正），默认是当前运行机器的处理器核数。这些参数先记住就可以了。
    sg ({0, 1}, optional) – 模型的训练算法: 1: skip-gram; 0: CBOW
    alpha (float, optional) – 初始学习率
    iter (int, optional) – 迭代次数，默认为5
"""
cut_dataset = np.concatenate((train["cut_q1"], train["cut_q2"]))  # 必须是切完后的列表，而不是拼成一个字符串
output = name + ".model"
model = Word2Vec(cut_dataset, sg=1, vector_size=400, min_count=2, window=5)
model.save(output)

In [28]:
output2 = "word2vec_" + name + ".vector"
model.save(output2)

In [29]:
word2vec_model = Word2Vec.load(output2)
k = word2vec_model.wv.index_to_key
sum([len(i) >= 2 for i in k])

32421

In [30]:
word2

,query1,query2,label,cut_q1,cut_q2
0,喜欢打篮球的男生喜欢什么样的女生,爱打篮球的男生喜欢什么样的女生,1,"[喜欢, 打篮球, 的, 男生, 喜欢, 什么样, 的, 女生]","[爱, 打篮球, 的, 男生, 喜欢, 什么样, 的, 女生]"
1,我手机丢了，我想换个手机,我想买个新手机，求推荐,1,"[我, 手机, 丢, 了, ，, 我想, 换个, 手机]","[我想, 买个, 新手机, ，, 求, 推荐]"
2,大家觉得她好看吗,大家觉得跑男好看吗？,0,"[大家, 觉得, 她, 好看, 吗]","[大家, 觉得, 跑, 男, 好看, 吗, ？]"
3,求秋色之空漫画全集,求秋色之空全集漫画,1,"[求, 秋色, 之空, 漫画, 全集]","[求, 秋色, 之空, 全集, 漫画]"
4,晚上睡觉带着耳机听音乐有什么害处吗？,孕妇可以戴耳机听音乐吗?,0,"[晚上, 睡觉, 带, 着, 耳机, 听, 音乐, 有, 什么, 害处, 吗, ？]","[孕妇, 可以, 戴, 耳机, 听, 音乐, 吗, ?]"
...,...,...,...,...,...
238761,女孩子说我是你的汤是什么意思,男孩给女孩说你的眼是海什么意思,0,"[女孩子, 说, 我, 是, 你, 的, 汤, 是, 什么, 意思]","[男孩, 给, 女孩, 说, 你, 的, 眼, 是, 海, 什么, 意思]"
238762,求重生之老公请接招全文,求重生之老公请接招>全文,1,"[求, 重生, 之, 老公, 请, 接招, 全文]","[求, 重生, 之, 老公, 请, 接招, >, 全文]"
238763,求小说电子书，,求《甄嬛》小说电子书！,0,"[求, 小说, 电子书, ，]","[求, 《, 甄, 嬛, 》, 小说, 电子书, ！]"
238764,杭州有什么好玩的地方？,杭州有什么好玩的地方求推,1,"[杭州, 有, 什么, 好玩, 的, 地方, ？]","[杭州, 有, 什么, 好玩, 的, 地方, 求, 推]"
